In [1]:
import requests 

#download course FAQ documents
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Initiate Connection to Elasticsearch

In [2]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '4d8a778da523', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'GgLK2U6tSWG2qPAi_mXilQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Index the Data

In [3]:

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
try:
    # Check if index already exists
    if not es.indices.exists(index=index_name):
        # Create the index with the specified settings and mappings
        response = es.indices.create(index=index_name, body=index_settings)
        print(f"Index '{index_name}' created successfully.")
    else:
        print(f"Index '{index_name}' already exists.")
except Exception as e:
    print(f"Unexpected error: {e}")

Index 'course-questions' created successfully.


In [4]:
#Index the documents
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [5]:
def retrieve_documents(query, index_name="course-questions", course_filter="", max_results=5):
    es = Elasticsearch("http://localhost:9200")
    
    # Construct the base query
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],  # Use only question and text fields and give question a boost of 4.
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    
    # Add the filter only if course_filter is not empty
    if course_filter:
        search_query["query"]["bool"]["filter"] = {
            "term": {
                "course": course_filter
            }
        }
    response = es.search(index=index_name, body=search_query)
    max_score = response['hits']['max_score']
    documents = [(hit['_source'], hit['_score']) for hit in response['hits']['hits']]
    return (documents, max_score)

## Ask a question and determine the max_score

In [6]:
question = "How do I execute a command in a running docker container?"

documents, max_score = retrieve_documents(question, max_results=5)

print(f"The max score is {max_score}")

for doc, score in documents:
    print(f"Doc score = {score}")
    print(f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n")

The max score is 83.73932
Doc score = 83.73932
Section: 5. Deploying Machine Learning Models
Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)


Doc score = 76.350395
Section: Module 1: Docker and Terraform
Question: PGCLI - running in a Docker container
Answer: In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.
Below the usage with values used in the videos of the course for:
network name (docker network)
postgres related variables for pgcli
Hostname
Username
Port
Database name
$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1
175dd47cda07:/# pgcli -h pg-databa

## Now limit the questions to 'machine-learning-zoomcamp'

In [7]:
question = "How do I execute a command in a running docker container?"

documents, max_score = retrieve_documents(question, max_results=3, course_filter='machine-learning-zoomcamp')

for doc, score in documents:
    print(f"Doc score = {score}")
    print(f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n")

Doc score = 83.73932
Section: 5. Deploying Machine Learning Models
Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)


Doc score = 51.000084
Section: 5. Deploying Machine Learning Models
Question: How do I copy files from my local machine to docker container?
Answer: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani


Doc score = 49.492252
Section: 5. Deployin

## Import OpenAI

First load the env variables from .envrc

In [8]:
from dotenv import load_dotenv
import os

# Load environment variables from .envrc
load_dotenv('.envrc')

True

In [18]:
from openai import OpenAI

True

In [10]:
client = OpenAI()

In [11]:
q = "How do I execute a command in a running docker container?"

response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

#get response from model
response.choices[0].message.content

'To execute a command in a running Docker container, you use the `docker exec` command. This command allows you to run a specified command within an already running container. Here’s the general syntax:\n\n```sh\ndocker exec [OPTIONS] CONTAINER COMMAND [ARG...]\n```\n\n### Common Examples\n\n1. **Run a simple command (e.g., `ls`):**\n\n   ```sh\n   docker exec CONTAINER_NAME_OR_ID ls\n   ```\n\n2. **Run an interactive shell (e.g., Bash):**\n\n   ```sh\n   docker exec -it CONTAINER_NAME_OR_ID /bin/bash\n   ```\n\n   or\n\n   ```sh\n   docker exec -it CONTAINER_NAME_OR_ID /bin/sh\n   ```\n\n   The `-i` flag stands for interactive, and the `-t` flag stands for TTY (teletypewriter), which together provide an interactive shell session.\n\n3. **Run a non-interactive command:**\n\n   ```sh\n   docker exec CONTAINER_NAME_OR_ID echo "Hello, World!"\n   ```\n\n### Options\n\n- `-i, --interactive`: Keeps STDIN open even if not attached.\n- `-t, --tty`: Allocates a pseudo-TTY.\n- `-d, --detach`: D

## Build a Prompt

Now we want to ask a question, but provide context as well.

In [12]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [13]:
context = ""

for doc, score in documents:
    context = context + f"Question: {doc['question']}\nAnswer: {doc['text']}\n\n"

print(context)

Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Question: How do I copy files from my local machine to docker container?
Answer: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Question: How do I copy files from a different folder into docker container’s working directory?
Answer: You can copy files from your local machine into a Docker container using t

In [14]:
q = "How do I execute a command in a running docker container?"
prompt = prompt_template.format(question=q, context=context).strip()
print(len(prompt))
print(prompt)

1498
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Question: How do I copy files from my local machine to docker container?
Answer: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/fi

## Get an Answer

We have the prompt and the context, now let's get an answer

In [20]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)

To execute a command in a running Docker container, follow these steps:

1. Identify the running container by finding its container ID using the `docker ps` command:
   ```bash
   docker ps
   ```

2. Once you have the container ID, use the `docker exec` command to execute the desired command inside the container. For example, to start a bash session in the container, use:
   ```bash
   docker exec -it <container-id> bash
   ```

Replace `<container-id>` with the actual container ID you identified in the first step.


## Tokens

When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

Let's calculate the number of tokens in our query:

In [22]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [24]:
print(f"Our prompt contains {len(encoding.encode(prompt))} tokens")

Our prompt contains 322 tokens
